In [68]:
import pandas as pd
import talib
import MetaTrader5 as mt5
import datetime

class Trend:
    
    def __init__(self):
        
        if not mt5.initialize(login=114781990, server="Exness-MT5Trial6",password="Mypassword$1234"):
            print("initialize() failed, error code =",mt5.last_error())
            quit()
        
        login = 114781990
        password = 'Mypassword$1234'
        server = 'Exness-MT5Trial6'

        mt5.login(login, password, server)

        rate = mt5.copy_rates_from('EURUSDm', mt5.TIMEFRAME_M15, datetime.datetime.now(), 10000)
        self.df = pd.DataFrame(rate)
        
         
        self.df['time']=pd.to_datetime(self.df['time'], unit='s')
        
        print(self.df.head())
        
    def display(self):
        self.df.to_excel('data.xlsx')
        
        
    def define_trend(self, current_high, current_low, prev_high, prev_low, days, i):
        #       Uptrend
            if current_high > prev_high and current_low > prev_low:
    #           Find point at which current sample
    #           crossed prev high

                stream = self.df.loc[self.df.index[i : i+days], 'high']
                up_candle_index = stream.loc[stream > prev_high].index[0]
                
    #             if df.loc[i+15, 'high'] > prev_high:
                    
                self.df.loc[up_candle_index : i+days, 'trend'] = 'up'

            
                if self.df.iloc[i-1]['trend'] == 'up':
                    self.df.loc[i: up_candle_index, 'trend'] = 'up'

                else:
                    self.df.loc[i: up_candle_index, 'trend'] = 'no trend'


    #         Downtrend
            elif (current_high < prev_high) and (current_low < prev_low):
    #           Find point at which current sample 
    #           crossed prev low

                stream = self.df.loc[self.df.index[i : i+days], 'low']
                down_candle_index = stream.loc[stream < prev_low,].index[0]
               
#                 print('DownCandleIndex: ', down_candle_index)
#                 print('Days Callee: ', days)
                
                self.df.loc[down_candle_index : i+days, 'trend'] = 'down'

                if self.df.iloc[i-1]['trend'] == 'down':
                    self.df.loc[i: down_candle_index, 'trend'] = 'down'

                else:
                    self.df.loc[i: down_candle_index, 'trend'] = 'no trend'


                    
            else:
                self.df.loc[i : i+days, 'trend'] = 'no trend'

    
    def get_trend(self, i=0):
        
        length = len(self.df)

        print('Length: ', length)
        
        days = 21

        prev_high = self.df.loc[self.df.index[i : i+days], 'high'].max()
        
        prev_low = self.df.loc[self.df.index[i : i+days], 'low'].min()

        while(i + days  < length):

            # print('In Trend !!')
        
            i += days
            
            current_high = self.df.loc[self.df.index[i : i+days], 'high'].max()
            current_low = self.df.loc[self.df.index[i : i+days], 'low'].min()
            
            self.define_trend(current_high, current_low, prev_high, prev_low, days, i)
            
            prev_high = current_high
            prev_low = current_low
            
                    
                
        if i < length-1:
            prev_high = self.df.loc[self.df.index[i-days:i], 'high'].max()
            prev_low = self.df.loc[self.df.index[i-days:i], 'low'].min()
            
            current_high = self.df.loc[self.df.index[i : length], 'high'].max()
            current_low = self.df.loc[self.df.index[i : length], 'low'].min()
            
            days = length-i
            
            self.define_trend(current_high, current_low, prev_high, prev_low, days, i)
            
            

In [69]:
ob = Trend()

                 time     open     high      low    close  tick_volume  \
0 2023-04-03 17:00:00  1.08882  1.08882  1.08797  1.08803          299   
1 2023-04-03 17:15:00  1.08803  1.08841  1.08803  1.08821          189   
2 2023-04-03 17:30:00  1.08822  1.08876  1.08817  1.08861          197   
3 2023-04-03 17:45:00  1.08860  1.08865  1.08825  1.08865          166   
4 2023-04-03 18:00:00  1.08862  1.08909  1.08855  1.08907          226   

   spread  real_volume  
0       8            0  
1       8            0  
2       8            0  
3       8            0  
4       8            0  


In [70]:
ob.get_trend()
ob.display()

Length:  10000
